In [1]:
import numpy as np
import networkx as nx
from sklearn.metrics import roc_auc_score
import pandas as pd
from google.colab import files

# Function to calculate the average shortest path length
def calculate_average_shortest_path(graph):
    try:
        return nx.average_shortest_path_length(graph)
    except nx.NetworkXError:
        return None

# Function to compute Asymmetric Mutual Influence (AMI)
def compute_asymmetric_mutual_influence(graph):
    nodes = list(graph.nodes())
    num_nodes = len(nodes)
    AMI = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j and graph.has_edge(nodes[i], nodes[j]):
                neighbors_i = set(graph.neighbors(nodes[i]))
                neighbors_j = set(graph.neighbors(nodes[j]))
                common_neighbors = neighbors_i & neighbors_j
                pij = len(common_neighbors) / num_nodes
                pi = len(neighbors_i) / num_nodes
                pj = len(neighbors_j) / num_nodes
                if pij > 0:
                    AMI[i, j] = pij * np.log(pij / (pi * pj))
    return AMI

# Function to compute transition matrix
def compute_transition_matrix(graph, AMI):
    nodes = list(graph.nodes())
    num_nodes = len(nodes)
    PR = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        neighbors = list(graph.neighbors(nodes[i]))
        influence_sum = sum(AMI[i, nodes.index(n)] for n in neighbors)
        for j in neighbors:
            j_idx = nodes.index(j)
            PR[i, j_idx] = AMI[i, j_idx] / influence_sum if influence_sum > 0 else 0
    return PR

# MIRW similarity computation
def MIRW_similarity(graph, PR, steps):
    nodes = list(graph.nodes())
    num_nodes = len(nodes)
    similarity = np.zeros((num_nodes, num_nodes))
    for t in range(steps):
        for i in range(num_nodes):
            for j in range(num_nodes):
                if i != j:
                    similarity[i, j] += PR[i, j] + PR[j, i]
    return similarity

# Evaluate Precision
def evaluate_precision(similarity, test_edges, L):
    scores = []
    for u, v in test_edges:
        scores.append((similarity[u, v], (u, v)))

    scores = sorted(scores, key=lambda x: x[0], reverse=True)
    top_L = scores[:L]
    correct_predictions = sum(1 for _, edge in top_L if edge in test_edges)
    precision = correct_predictions / L
    return precision

# Evaluate AUC
def evaluate_AUC(graph, similarity, test_edges, non_edges):
    scores = []
    labels = []

    for u, v in test_edges:
        scores.append(similarity[u, v])
        labels.append(1)

    for u, v in non_edges:
        scores.append(similarity[u, v])
        labels.append(0)

    auc = roc_auc_score(labels, scores)
    return auc

# MIRW Algorithm
def MIRW_algorithm(graph, L, test_size=0.1, num_iterations=10):
    auc_scores = []
    precision_scores = []

    for _ in range(num_iterations):
        edges = list(graph.edges())
        np.random.shuffle(edges)
        num_test = int(len(edges) * test_size)
        test_edges = edges[:num_test]
        train_edges = edges[num_test:]

        train_graph = nx.Graph()
        train_graph.add_edges_from(train_edges)

        non_edges = list(nx.non_edges(train_graph))
        np.random.shuffle(non_edges)
        non_edges = non_edges[:len(test_edges)]

        non_observed_edges = test_edges + non_edges

        node_to_index = {node: i for i, node in enumerate(train_graph.nodes())}

        AMI_matrix = compute_asymmetric_mutual_influence(train_graph)
        PR_matrix = compute_transition_matrix(train_graph, AMI_matrix)

        avg_shortest_path = calculate_average_shortest_path(train_graph)
        steps = int(avg_shortest_path) if avg_shortest_path is not None else 3

        similarity_scores = MIRW_similarity(train_graph, PR_matrix, steps)

        non_observed_edges_idx = []
        for u, v in non_observed_edges:
            if u in node_to_index and v in node_to_index:
                non_observed_edges_idx.append((node_to_index[u], node_to_index[v]))

        auc = evaluate_AUC(train_graph, similarity_scores, non_observed_edges_idx, non_observed_edges_idx)
        auc_scores.append(auc)

        precision = evaluate_precision(similarity_scores, non_observed_edges_idx, L)
        precision_scores.append(precision)

    avg_auc = np.mean(auc_scores)
    avg_precision = np.mean(precision_scores)

    return avg_auc, avg_precision

# Dataset 1: Karate Club
G_karate = nx.karate_club_graph()
auc_karate, precision_karate = MIRW_algorithm(G_karate, L=20)
print("Karate Club Dataset:")
print("Number of nodes:", G_karate.number_of_nodes())
print("Number of edges:", G_karate.number_of_edges())
print(f"Average AUC Score: {auc_karate}")
print(f"Average Precision@20: {precision_karate}")

# Dataset 2: Yeast Dataset
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
columns = ["Sequence_Name", "mcg", "gvh", "alm", "mit", "erl", "pox", "vac", "nuc", "Class"]
data = pd.read_csv(file_name, sep='\s+', names=columns)

G_yeast = nx.Graph()

for i, row in data.iterrows():
    G_yeast.add_node(row["Sequence_Name"], features=row[1:-1].tolist(), label=row["Class"])

for i, row1 in data.iterrows():
    for j, row2 in data.iterrows():
        if i < j and row1["Class"] == row2["Class"]:
            G_yeast.add_edge(row1["Sequence_Name"], row2["Sequence_Name"])

auc_yeast, precision_yeast = MIRW_algorithm(G_yeast, L=100)
print("\nYeast Dataset:")
print("Number of nodes:", G_yeast.number_of_nodes())
print("Number of edges:", G_yeast.number_of_edges())
print(f"Average AUC Score: {auc_yeast}")
print(f"Average Precision@100: {precision_yeast}")

# Create a DataFrame to store the results
results = pd.DataFrame({
    "Dataset": ["Karate Club", "Yeast"],
    "Number of Nodes": [G_karate.number_of_nodes(), G_yeast.number_of_nodes()],
    "Number of Edges": [G_karate.number_of_edges(), G_yeast.number_of_edges()],
    "Average AUC": [auc_karate, auc_yeast],
    "Average Precision": [precision_karate, precision_yeast]
})

# Display the results as a table
print("\n\nResults Table:")
print(results)


Karate Club Dataset:
Number of nodes: 34
Number of edges: 78
Average AUC Score: 0.5
Average Precision@20: 0.6849999999999999


Saving yeast.data to yeast (4).data

Yeast Dataset:
Number of nodes: 1462
Number of edges: 235193
Average AUC Score: 0.5
Average Precision@100: 1.0


Results Table:
       Dataset  Number of Nodes  Number of Edges  Average AUC  \
0  Karate Club               34               78          0.5   
1        Yeast             1462           235193          0.5   

   Average Precision  
0              0.685  
1              1.000  
